# Ontology Search

This program will load and explore the Cell Ontology from https://www.ebi.ac.uk/ols/ontologies/cl

From the above site, you can download the ontology as *cl.owl*. To access the ontology, we will use OwlReady2, which can be installed with *pip*. 

- Documentation: https://owlready2.readthedocs.io/en/v0.37/index.html
- Original paper: http://www.lesfleursdunormal.fr/static/_downloads/article_owlready_aim_2017.pdf
- Citation: Lamy, JB. Owlready: Ontology-oriented programming in Python with automatic classification and high level constructs for biomedical ontologies. Artificial Intelligence In Medicine 2017;80:11-28

In [1]:
import owlready2 as owl

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [6]:
# load the ontology

onto = owl.get_ontology('file://cl.owl').load()

In [7]:
onto

get_ontology("http://purl.obolibrary.org/obo/cl.owl#")

In [10]:
list(onto.classes())

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://purl.obolibrary.org/obo/BFO_0000023
  http://purl.obolibrary.org/obo/CHEBI_50906

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://purl.obolibrary.org/obo/BFO_0000023
  http://purl.obolibrary.org/obo/CHEBI_50906

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://purl.obolibrary.org/obo/CHEBI_36080
  http://purl.obolibrary.org/obo/PR_000000001

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://purl.obolibrary.org/obo/CHEBI_36080
  http://purl.obolibrary.org/obo/PR_000000001



[obo.CL_0000000,
 obo.BFO_0000002,
 obo.BFO_0000003,
 obo.BFO_0000004,
 obo.BFO_0000017,
 obo.BFO_0000019,
 obo.BFO_0000020,
 obo.BFO_0000031,
 obo.BFO_0000034,
 obo.BFO_0000023,
 obo.BFO_0000016,
 obo.BFO_0000040,
 obo.BFO_0000006,
 obo.CARO_0000003,
 obo.BFO_0000141,
 obo.CARO_0000010,
 obo.CARO_0001001,
 obo.PATO_0002124,
 obo.BFO_0000015,
 obo.CL_0000540,
 obo.CARO_0000006,
 obo.CARO_0001010,
 obo.RO_0002577,
 obo.UBERON_0001637,
 obo.UBERON_0001638,
 obo.CARO_0010000,
 obo.CARO_0000000,
 obo.GO_0008150,
 obo.PATO_0000402,
 obo.UBERON_0000475,
 obo.UBERON_0010912,
 obo.CL_0000101,
 obo.PATO_0000001,
 obo.UBERON_0004111,
 obo.UBERON_0000463,
 obo.BFO_0000009,
 obo.CHEBI_50906,
 obo.CARO_0030000,
 obo.CARO_0000007,
 obo.PATO_0002266,
 obo.PATO_0010000,
 obo.CARO_0000004,
 obo.CARO_0001008,
 obo.PATO_0000125,
 obo.CARO_0000012,
 obo.CARO_0010004,
 obo.CARO_0000013,
 obo.CARO_0020003,
 obo.CARO_0000014,
 obo.CARO_0000040,
 obo.CARO_0000042,
 obo.CARO_0001000,
 obo.CARO_0000032,
 obo.PA

In [14]:
print(onto.metadata.comment)

['See PMID:15693950, PMID:12799354, PMID:20123131, PMID:21208450; Contact Alexander Diehl, addiehl@buffalo.edu, University at Buffalo.', 'See PMID:15693950, PMID:12799354, PMID:20123131, PMID:21208450; Contact Alexander Diehl, addiehl@buffalo.edu, University at Buffalo.']


In [24]:
print(onto.get_properties)

None
